In [4]:
%run 10_ngram_common.ipynb

In [5]:
normal_speakers = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

# read libra

In [6]:
libra = load_dataset("librispeech_asr", 'clean', cache_dir="/work/van-speech-nlp/cache")

Generating train.100 split:   0%|          | 0/28539 [00:00<?, ? examples/s]

Generating train.360 split:   0%|          | 0/104014 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2703 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2620 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/45 [00:00<?, ?it/s]

In [15]:
translations=libra['train.360']['text']
translations[:3]

['THE SECOND IN IMPORTANCE IS AS FOLLOWS SOVEREIGNTY MAY BE DEFINED TO BE THE RIGHT OF MAKING LAWS IN FRANCE THE KING REALLY EXERCISES A PORTION OF THE SOVEREIGN POWER SINCE THE LAWS HAVE NO WEIGHT',
 'TILL HE HAS GIVEN HIS ASSENT TO THEM HE IS MOREOVER THE EXECUTOR OF ALL THEY ORDAIN THE PRESIDENT IS ALSO THE EXECUTOR OF THE LAWS BUT HE DOES NOT REALLY CO OPERATE IN THEIR FORMATION',
 'SINCE THE REFUSAL OF HIS ASSENT DOES NOT ANNUL THEM HE IS THEREFORE MERELY TO BE CONSIDERED AS THE AGENT OF THE SOVEREIGN POWER BUT NOT ONLY DOES THE KING OF FRANCE EXERCISE A PORTION OF THE SOVEREIGN POWER']

In [16]:
cleaned_translations = []

for text in translations:
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    cleaned_translations.append(cleaned_text)

In [17]:
cleaned_translations[:3]

['the second in importance is as follows sovereignty may be defined to be the right of making laws in france the king really exercises a portion of the sovereign power since the laws have no weight',
 'till he has given his assent to them he is moreover the executor of all they ordain the president is also the executor of the laws but he does not really co operate in their formation',
 'since the refusal of his assent does not annul them he is therefore merely to be considered as the agent of the sovereign power but not only does the king of france exercise a portion of the sovereign power']

In [20]:
with open("DATASET_libri.txt", "w") as file:
    for text in cleaned_translations:
        file.write(text + "\n")

In [18]:
word_set_europarl = set()

for sentence in cleaned_translations:
    words = sentence.split()
    word_set_europarl.update(words)

In [19]:
len(word_set_europarl)

58354

# train LM with LIBRA

In [21]:
!kenlm/build/bin/lmplz -o 3 < DATASET_libri.txt > libri.apra --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/DATASET_libri.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 3595494 types 58357
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:700284 2:55979069440 3:104960753664
Statistics:
1 58357 D1=0.578511 D2=0.988481 D3+=1.43628
2 986830 D1=0.758182 D2=1.0831 D3+=1.37103
3 2458575 D1=0.857684 D2=1.17924 D3+=1.35938
Memory estimate for binary LM:
type    MB
probing 66 assuming -p 1.5
probing 72 assuming -r models -p 1.5
trie    26 without quantization
trie    14 assuming -q 8 -b 8 quantization 
trie    25 assuming -a 22 array pointer compression
trie    13 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:700

# read torgo

In [22]:
# Read the dataset
data_df = pd.read_csv('torgo.csv')
dataset_csv = load_dataset('csv', data_files='torgo.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [23]:
dataset_csv

DatasetDict({
    train: Dataset({
        features: ['session', 'audio', 'text', 'speaker_id'],
        num_rows: 16394
    })
})

In [28]:
all_texts = dataset_csv['train']['text']

cleaned_texts = []
for text in all_texts:
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    cleaned_texts.append(cleaned_text)

In [26]:
cleaned_texts[:10]

['stick',
 'except in the winter when the ooze or snow or ice prevents',
 'pat',
 'up',
 'meat',
 'meat',
 'know',
 'he slowly takes a short walk in the open air each day',
 'air',
 'swarm']

In [30]:
from collections import defaultdict

speaker_texts = defaultdict(list)

for i in range(len(data_df)):
    speaker_id = data_df.loc[i, 'speaker_id']
    # print(speaker_id)
    if speaker_id not in normal_speakers:
        text = data_df.loc[i, 'text']
        speaker_texts[speaker_id].append(text)


for speaker_id, texts in speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")
    # for text in texts:
    #     print(text)
    # print()

F01: 228
F03: 1075
F04: 667
M01: 739
M02: 766
M03: 800
M04: 652
M05: 573


# calculate unique words

In [31]:
unique_words_per_speaker = {}

for speaker_id, texts in speaker_texts.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 190 unique words
F03: 768 unique words
F04: 669 unique words
M01: 605 unique words
M02: 622 unique words
M03: 645 unique words
M04: 450 unique words
M05: 718 unique words


# vocabulary similarity

In [32]:
out_of_domain_words_per_speaker = {}

for speaker_id, unique_words in unique_words_per_speaker.items():
    overlap = len(unique_words & word_set_europarl)
    similarity = overlap / len(unique_words) if len(unique_words) > 0 else 0
    out_of_domain_similarity = (1 - similarity)*100

    out_of_domain_words = {word for word in unique_words if word not in word_set_europarl}
    out_of_domain_words_per_speaker[speaker_id] = out_of_domain_words

    print(f"{speaker_id}: {out_of_domain_similarity:.2f}% dissimilarity with word_set_europarl")
    print(f"{speaker_id}: out_of_domain_words count: {len(out_of_domain_words)}")
    print(f"Out of domain words for {speaker_id}: {out_of_domain_words}")
    print()

F01: 0.53% dissimilarity with word_set_europarl
F01: out_of_domain_words count: 1
Out of domain words for F01: {'gadget'}

F03: 2.21% dissimilarity with word_set_europarl
F03: out_of_domain_words count: 17
Out of domain words for F03: {'gadget', 'validated', 'ninetythree', 'whitecapped', 'shimmers', 'tort', 'mut', 'sews', 'cupping', 'advisement', 'foxtrot', 'snoop', 'tango', 'kilo', 'treehouse', 'sweaters', 'exam'}

F04: 2.69% dissimilarity with word_set_europarl
F04: out_of_domain_words count: 18
Out of domain words for F04: {'gadget', 'xray', 'ninetythree', 'validated', 'shimmers', 'mut', 'lilyrare', 'sews', 'ticker', 'leaks', 'advisement', 'bloat', 'foxtrot', 'snoop', 'kilo', 'treehouse', 'sweaters', 'exam'}

M01: 2.48% dissimilarity with word_set_europarl
M01: out_of_domain_words count: 15
Out of domain words for M01: {'gadget', 'validated', 'ninetythree', 'shimmers', 'mut', 'sews', 'ticker', 'advisement', 'bloat', 'foxtrot', 'snoop', 'kilo', 'treehouse', 'sweaters', 'exam'}

M02: 

# Out of domain analysis

In [33]:
import kenlm

model=kenlm.Model("libri.apra") 
per=model.perplexity("your text sentance")
print(per)

per=model.perplexity("resumption of the session")
print(per)

10139.860730126507
482.3399492347551


Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/libri.apra
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [34]:
# Calculate perplexity for each text
perplexities = []
for text in cleaned_texts:
    perplexity = model.perplexity(text)
    perplexities.append(perplexity)

# Calculate the average perplexity
avg_perplexity = np.mean(perplexities)
print(f"Average Perplexity: {avg_perplexity}")

Average Perplexity: 3979.8411851554615


# clean the text

In [35]:
import re

cleaned_speaker_texts = {}

for speaker_id, texts in speaker_texts.items():
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
    cleaned_speaker_texts[speaker_id] = cleaned_texts

for speaker_id, texts in cleaned_speaker_texts.items():
    print(f"Speaker {speaker_id}:")
    for text in texts[:2]:  
        print(text)
    print()

Speaker F01:
stick
except in the winter when the ooze or snow or ice prevents

Speaker F03:
beta
stubble

Speaker F04:
fee
yet he still thinks as swiftly as ever

Speaker M01:
when he speaks his voice is just a bit cracked and quivers a trifle
trait

Speaker M02:
grow
feed

Speaker M03:
now
know

Speaker M04:
trouble
spark

Speaker M05:
swore
train



In [36]:
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

Speaker F01: Average Perplexity: 3321.6634825919314
Speaker F03: Average Perplexity: 3828.1590222953987
Speaker F04: Average Perplexity: 4077.993261310603
Speaker M01: Average Perplexity: 3917.8475900790772
Speaker M02: Average Perplexity: 3862.7596868415703
Speaker M03: Average Perplexity: 3824.284655645186
Speaker M04: Average Perplexity: 3908.6444183261924
Speaker M05: Average Perplexity: 4063.3047017015047


# output unigram

In [44]:
arpa_file = f"libri_lm_arpa_files/libri.apra"
output_file = f"libri_lm_arpa_files/unigram.txt"

unigrams = []

with open(arpa_file, "r") as file:
    start_extraction = False
    for line in file:
        line = line.strip()
        if start_extraction:
            if not line:  
                break
            parts = line.split("\t")
            unigram = parts[1]  
            unigrams.append(unigram)
        elif line.startswith("\\1-grams:"):
            start_extraction = True

with open(output_file, "w") as file:
    for unigram in unigrams:
        file.write(unigram + "\n")

libri.apra
